<a href="https://colab.research.google.com/github/DongChyeon/Tensorflow-Study/blob/main/%ED%85%90%EC%84%9C%ED%94%8C%EB%A1%9C%EC%9A%B0%20%EB%9D%BC%EC%9D%B4%ED%8A%B8%20%EC%88%AB%EC%9E%90%20%EA%B5%AC%EB%B6%84%EA%B8%B0%20%EC%98%88%EC%8B%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>사용할 라이브러리 임포트</h3>

In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
import random

<h3>keras가 제공하는 MNIST 데이터셋 다운로드</h3>

In [ ]:
# keras가 제공하는 MNiST 데이터셋을 다운받고, 이를 훈련용 데이터셋과 테스트용 데이터셋으로 나눔
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data();

# 이미지 픽셀값을 0과 1로 정규화함
train_images = train_images / 255.0
test_images = test_images / 255.0

# 훈련용 데이터셋의 처음 25개의 이미지를 보여줌
plt.figure(figsize=(10,10))
for i in range(25):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(train_images[i], cmap=plt.cm.gray)
  plt.xlabel(train_labels[i])
plt.show()

<h3>숫자 이미지 판별 모델 학습</h3>

In [ ]:
# 모델 아키텍처 정의
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28,28)),
  keras.layers.Reshape(target_shape=(28,28,1)),
  keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation=tf.nn.relu),
  keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation=tf.nn.relu),
  keras.layers.MaxPooling2D(pool_size=(2,2)),
  keras.layers.Dropout(0.25),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# 모델을 어떻게 학습시킬지 정의
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# 숫자 분류 모델 학습
model.fit(train_images, train_labels, epochs=5)

model.summary()

<h3>모델 평가</h3>

In [ ]:
# 테스트 데이터셋을 이용한 모델 평가
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy : ', test_acc)

def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

predictions = model.predict(test_images)

prediction_digits = np.argmax(predictions, axis=1)

plt.figure(figsize=(18,18))
for i in range(100):
  ax = plt.subplot(10,10,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  image_index = random.randint(0,len(prediction_digits))
  plt.imshow(test_images[image_index], cmap=plt.cm.gray)
  ax.xaxis.label.set_color(get_label_color(prediction_digits[image_index],test_labels[image_index]))
  plt.xlabel('Predicted : %d' % prediction_digits[image_index])
plt.show()

<h3>keras 모델을 Tensorflow Lite 모델로 변환</h3>

In [ ]:
# keras 모델을 TF lite 모델로 변환
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_float_model = converter.convert()

# 모델의 크기를 KB 단위로 보여줌
float_model_size = len(tflite_float_model) / 1024
print('Float model size = %dKBs' % float_model_size)

# 양자화를 활용해서 한 번 더 변환 (용량을 더 줄여줌)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

# 모델의 크기를 비교
quantized_model_size = len(tflite_quantized_model) / 1024
print('Quantized model size = %dKBs' % quantized_model_size)
print('which is about %d%% of the float model size.' % (quantized_model_size * 100 / float_model_size))

<h3>Tensorflow Lite 모델 평가</h3>

In [ ]:
def evaluate_tflite_model(tflite_model):
  interpreter = tf.lite.Interpreter(model_content=tflite_model)
  interpreter.allocate_tensors()
  input_tensor_index = interpreter.get_input_details()[0]["index"]
  output = interpreter.tensor(interpreter.get_output_details()[0]["index"])

  # 테스트 데이터셋의 모든 이미지에 대해 추론 수행
  prediction_digits = []
  for test_image in test_images:
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_tensor_index, test_image)

    interpreter.invoke()

    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  accurate_count = 0
  for index in range(len(prediction_digits)):
    if prediction_digits[index] == test_labels[index]:
      accurate_count += 1
  accuracy = accurate_count * 1.0 / len(prediction_digits)
  
  return accuracy

# 모델의 정확도를 보여줌
float_accuracy = evaluate_tflite_model(tflite_float_model)
print('Float model accuracy = %.4f' % float_accuracy)

# 양자화 모델과의 정확도를 비교
quantized_accuracy = evaluate_tflite_model(tflite_quantized_model)
print('Quantized model accuracy = %.4f' % quantized_accuracy)
print('Accuracy drop = %.4f' % (float_accuracy - quantized_accuracy))

<h3>모델 다운로드</h3>

In [ ]:
# 양자화 모델을 다운로드
f = open('mnist.tflite', "wb")
f.write(tflite_quantized_model)
f.close()

from google.colab import files
files.download('mnist.tflite')

print('`mnist.tflite` has been downloaded')

자세한 내용은 https://colab.research.google.com/github/tensorflow/examples/blob/master/lite/codelabs/digit_classifier/ml/step2_train_ml_model.ipynb?hl=ko#scrollTo=r0WlLrJcnwWp 에...